In [1]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools import tool
from dotenv import load_dotenv
import os
from pydantic import BaseModel
from typing import Any, Dict
import json

from maxim import Maxim

print("✅ All dependencies imported successfully!")

load_dotenv()

✅ All dependencies imported successfully!


True

In [ ]:
# Initialize Maxim logger if credentials are provided

logger = Maxim({}).logger({})

# Instrumenting Agno
from maxim.logger.agno import instrument_agno

instrument_agno(logger)

[MaximSDK] Initializing Maxim AI(v3.9.6)
[MaximSDK] Using info logging level.
[MaximSDK] For debug logs, set global logging level to debug logging.basicConfig(level=logging.DEBUG).


[MaximSDK] Key 'model' is not serializable and will be removed from data
[MaximSDK] Key 'model' is not serializable and will be removed from data
[MaximSDK] Key 'model' is not serializable and will be removed from data
[MaximSDK] Key 'model' is not serializable and will be removed from data


In [4]:
# Setup OpenAI model
llm = OpenAIChat(name="gpt-4o-mini", api_key=os.environ["OPENAI_API_KEY"])

print("✅ OpenAI model configured successfully!")

✅ OpenAI model configured successfully!


In [3]:
# Pydantic models for structured output
class ExtractFormat(BaseModel):
    """Model for extracted recipe information"""
    dish_name: str = ""
    number_served: int = 5
    file_name: str = ""

class ChefFormat(BaseModel):
    """Model for recipe data"""
    recipe_data: str

print("✅ Data models defined successfully!")


✅ Data models defined successfully!


In [5]:
# File writing tool
@tool(show_result=True)
def write_file(filename: str, content: str) -> str:
    """Write content to a file."""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"✅ Successfully wrote content to {filename}"
    except Exception as e:
        return f"❌ Error writing to file: {str(e)}"

print("✅ Custom tools created successfully!")


✅ Custom tools created successfully!


In [6]:
# Extraction Agent
extraction_agent = Agent(
    name="Extraction Agent",
    role="Extract the names of dishes, files, and the quantity or numbers from given input",
    model=llm,
    instructions=[
        "You are expert in structuring any prose, and extracting information",
        "You are brief and to the point",
        "Extract dish_name, number_served, and file_name from the user input",
        "Return the information in JSON format"
    ],
    description="Extract details like dish_name, file_name and number of people to be served",
    response_model=ExtractFormat,
    markdown=True,
    show_tool_calls=True
)

print("✅ Extraction Agent created successfully!")
print(f"📝 Agent Name: {extraction_agent.name}")
print(f"🎯 Agent Role: {extraction_agent.role}")


✅ Extraction Agent created successfully!
📝 Agent Name: Extraction Agent
🎯 Agent Role: Extract the names of dishes, files, and the quantity or numbers from given input


In [7]:
# Chef Agent
chef_agent = Agent(
    name="Chef Agent", 
    role="Write recipes for the dishes asked by the users",
    model=llm,
    tools=[write_file],
    instructions=[
        "You are a Michelin star rated Master chef with culinary skills ranging from western to eastern region",
        "You are brief and to the point",
        "Provide short to the point recipes with quantity of ingredients",
        "Write step by step guide for making the dish",
        "Format the recipe in markdown",
        "If a filename is provided, use the write_file tool to save the recipe"
    ],
    description="Provide recipes with ingredients and cooking instructions",
    response_model=ChefFormat,
    markdown=True,
    show_tool_calls=True
)

print("✅ Chef Agent created successfully!")
print(f"📝 Agent Name: {chef_agent.name}")
print(f"🎯 Agent Role: {chef_agent.role}")
print(f"🛠️ Available Tools: {len(chef_agent.tools)} tool(s)")


✅ Chef Agent created successfully!
📝 Agent Name: Chef Agent
🎯 Agent Role: Write recipes for the dishes asked by the users
🛠️ Available Tools: 1 tool(s)


In [8]:
# Callback functions for monitoring
def on_extraction_complete(result: Dict[str, Any], **kwargs):
    """Callback for extraction completion"""
    print("\n" + "="*30)
    print("🔍 EXTRACTION COMPLETE")
    print("="*30)
    print(f"📊 Extraction Result: {result}")
    
    if hasattr(result, 'dish_name'):
        # It's a Pydantic model
        print(f"🍽️  Dish Name: {result.dish_name or 'N/A'}")
        print(f"👥 Number to serve: {result.number_served or 'N/A'}")
        print(f"📁 File Name: {result.file_name or 'N/A'}")
    elif isinstance(result, dict):
        # It's a dictionary
        print(f"🍽️  Dish Name: {result.get('dish_name', 'N/A')}")
        print(f"👥 Number to serve: {result.get('number_served', 'N/A')}")
        print(f"📁 File Name: {result.get('file_name', 'N/A')}")
    print("="*30)

def on_recipe_complete(result: Dict[str, Any], **kwargs):
    """Callback for recipe completion"""
    print("\n" + "="*30)
    print("👨‍🍳 RECIPE COMPLETE")
    print("="*30)
    print("🎉 Recipe has been generated successfully!")
    print("="*30)

print("✅ Callback functions defined successfully!")


✅ Callback functions defined successfully!


In [9]:
class CookingAgentTeam:
    """Cooking agent team that handles extraction and recipe generation"""
    
    def __init__(self):
        self.extraction_agent = extraction_agent
        self.chef_agent = chef_agent
        print("🤝 CookingAgentTeam initialized!")
        
    def process_request(self, user_input: str) -> Dict[str, Any]:
        """Process cooking request through extraction and recipe generation"""
        
        print(f"📝 Processing request: {user_input}")
        print("-" * 50)
        
        # Step 1: Extract information
        print("🔍 Step 1: Extracting information from request...")
        extraction_result = self.extraction_agent.run(user_input)
        
        # Handle the result properly
        if hasattr(extraction_result, 'content'):
            extracted_data = extraction_result.content
        else:
            extracted_data = extraction_result
            
        # Parse JSON if it's a string
        if isinstance(extracted_data, str):
            try:
                extracted_data = json.loads(extracted_data)
            except json.JSONDecodeError:
                extracted_data = {
                    "dish_name": "Unknown Dish",
                    "number_served": 4,
                    "file_name": ""
                }
        
        on_extraction_complete(extracted_data)
        
        # Step 2: Generate recipe
        print("👨‍🍳 Step 2: Generating recipe...")
        
        # Handle both Pydantic model and dictionary cases
        if hasattr(extracted_data, 'dish_name'):
            # It's a Pydantic model
            dish_name = extracted_data.dish_name or 'the requested dish'
            number_served = extracted_data.number_served or 4
            file_name = extracted_data.file_name or ""
        else:
            # It's a dictionary (fallback)
            dish_name = extracted_data.get('dish_name', 'the requested dish')
            number_served = extracted_data.get('number_served', 4)
            file_name = extracted_data.get('file_name', "")
        
        recipe_prompt = f"""
        Create a detailed recipe for {dish_name} 
        to serve {number_served} people.
        
        Include:
        - List of ingredients with precise quantities
        - Step-by-step cooking instructions
        - Cooking time and difficulty level
        - Any special tips or variations
        
        Format the recipe in markdown.
        
        {f"Save the recipe to file: {file_name}" if file_name else ""}
        """
        
        recipe_result = self.chef_agent.run(recipe_prompt)
        
        # Handle the result properly
        if hasattr(recipe_result, 'content'):
            recipe_data = recipe_result.content
        else:
            recipe_data = recipe_result
            
        on_recipe_complete(recipe_data)
        
        return {
            "extraction": extracted_data,
            "recipe": recipe_data,
            "status": "completed"
        }

# Initialize the cooking team
cooking_team = CookingAgentTeam()
print("✅ Cooking Agent Team ready for action!")


🤝 CookingAgentTeam initialized!
✅ Cooking Agent Team ready for action!


In [11]:
def test_cooking_agent():
    """Test function to demonstrate the cooking agent functionality"""
    
    # Test case 1: Simple recipe request
    test_input = "I want a recipe for Spaghetti Carbonara to serve 4 people save to ./carbonara.md"
    
    print("🍝 Testing Cooking Agent...")
    print("="*60)
    print(f"📝 Input: {test_input}")
    print("="*60)
    
    # Process the request
    result = cooking_team.process_request(test_input)
    
    print("\n" + "="*60)
    print("📋 FINAL RESULT")
    print("="*60)
    print(f"✅ Status: {result['status']}")
    print(f"📊 Extracted Info: {result['extraction']}")
    print(f"📝 Recipe Generated: {'✅' if result['recipe'] else '❌'}")
    print("="*60)
    
    return result

# Run the test
result = test_cooking_agent()
print("\n🎉 Cooking agent test completed successfully!")


🍝 Testing Cooking Agent...
📝 Input: I want a recipe for Spaghetti Carbonara to serve 4 people save to ./carbonara.md
📝 Processing request: I want a recipe for Spaghetti Carbonara to serve 4 people save to ./carbonara.md
--------------------------------------------------
🔍 Step 1: Extracting information from request...

🔍 EXTRACTION COMPLETE
📊 Extraction Result: dish_name='Spaghetti Carbonara' number_served=4 file_name='./carbonara.md'
🍽️  Dish Name: Spaghetti Carbonara
👥 Number to serve: 4
📁 File Name: ./carbonara.md
👨‍🍳 Step 2: Generating recipe...


WARNING  Failed to parse cleaned JSON: Expecting value: line 1 column 1 (char 0)


👨‍🍳 RECIPE COMPLETE
🎉 Recipe has been generated successfully!

📋 FINAL RESULT
✅ Status: completed
📊 Extracted Info: dish_name='Spaghetti Carbonara' number_served=4 file_name='./carbonara.md'
📝 Recipe Generated: ✅

🎉 Cooking agent test completed successfully!


In [ ]:
# 🎯 Try different recipe requests!
# Modify this variable and run the cell to test different recipes

user_request = "Give me a recipe for Chicken Tikka Masala for 6 people save to tikka_masala.md"

# Other examples you can try:
# user_request = "I need a simple recipe for Chocolate Pancakes for 2 people"
# user_request = "How do I make Beef Tacos for 8 people save to tacos.md"
# user_request = "Recipe for Vegetarian Pasta Primavera serving 5"
# user_request = "I want to make Homemade Pizza for 4 people save to pizza_recipe.md"

print(f"🍽️ Processing: {user_request}")
print("-" * 70)

# Process the request
custom_result = cooking_team.process_request(user_request)

extraction_data = custom_result['extraction']

# Handle both Pydantic model and dictionary cases
if hasattr(extraction_data, 'dish_name'):
    # It's a Pydantic model
    dish_name = extraction_data.dish_name
    number_served = extraction_data.number_served
    file_name = extraction_data.file_name
else:
    # It's a dictionary
    dish_name = extraction_data.get('dish_name', 'Unknown')
    number_served = extraction_data.get('number_served', 'Unknown')
    file_name = extraction_data.get('file_name', '')

print(f"\n✨ Recipe for {dish_name} is ready!")
print(f"👥 Serves: {number_served} people")

if file_name:
    print(f"📁 Saved to: {file_name}")
else:
    print("📁 No file specified - recipe displayed only")
